In [2]:
import os
import sys
import zipfile
import subprocess
import pandas as pd
from datetime import date, timedelta
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [2]:
sys.path.append('/Users/jangfeller/Library/Mobile Documents/com~apple~CloudDocs/_Ironhack/projects/mid_project/app/utils/')
from functions import get_latest_master, create_ML1_df, create_ML1_df_training, run_xgb

In [3]:
step_1 = get_latest_master()

Dataset URL: https://www.kaggle.com/datasets/benjaminpo/s-and-p-500-with-dividends-and-splits-daily-updated
License(s): CC-BY-SA-4.0
s-and-p-500-with-dividends-and-splits-daily-updated.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
ml1 = create_ML1_df(step_1)

In [5]:
ml1_training = create_ML1_df_training(step_1)

In [6]:
tipps = run_xgb(ml1, ml1_training)

In [ ]:
def run_xgb(df4, df4_training):

    """ creates df with top 20 predicted gains """

    def XGB_train_real():

        # short cleaning
        df_x = df4_training.copy().reset_index()
        df_x.drop(columns=["company", "sector", "subsector"], inplace=True)
        df_x.set_index("date", inplace=True)
        df_x.sort_index(inplace=True)

        # training with data only until training_end
        xg_df = df_x.copy()

        # X y split
        X = xg_df.drop(columns="price_30d")
        y = xg_df["price_30d"]

        # normalization
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)

        # train model
        xgbr = xgb.XGBRFRegressor()
        xgbr.fit(X_scaled, y)

        return xgbr



    def get_stocks():

        """
        Output: df with top 20 gainers
        """

        yesterday = date.today() - timedelta(days=2)
        df_ga = df4.copy()
        df_ga.reset_index(inplace=True)
        df_ga["date"] = pd.to_datetime(df_ga["date"])
        gains_df = pd.DataFrame(columns=["company", "close", "prediction", "gain_predicted"])
        df_ga.drop(columns=["sector", "subsector"], inplace=True)
        df_ga.set_index("date", inplace=True)    
        df_ga.sort_index(inplace=True)

        xg_df = df_ga.loc[str(yesterday)]
        xg_df.sort_index(inplace=True)

        X = xg_df.drop(columns=["company"])
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)
        pred_xgb = xgbr.predict(X_scaled)
        test = xg_df[["company", "close"]]
        test["prediction"] = pred_xgb
        test["gain_predicted"] = (test["prediction"] - test["close"]) / test["close"] * 100
        test.sort_values(by="gain_predicted", ascending=False, inplace=True)
        test.reset_index()
        #test.drop(columns="date")
        test.set_index("company")

        return test

    

    xgbr = XGB_train_real()
    stocks = get_stocks()
    stocks.to_csv("top_20.csv")
    return stocks

In [3]:
df = pd.read_csv("top_20.csv")

In [4]:
df

,date,close,company,prediction,gain_predicted
0,2024-12-12,782.349976,LLY,1697.081500,116.921019
1,2024-12-12,740.969971,REGN,1524.626500,105.760898
2,2024-12-12,786.590027,URI,1524.626500,93.827332
3,2024-12-12,925.549988,NFLX,1653.107900,78.608172
4,2024-12-12,988.390015,COST,1715.096600,73.524270
...,...,...,...,...,...
494,2024-12-12,12.490000,WBD,2.845127,-77.220761
495,2024-12-12,9.730000,WBA,1.264746,-87.001581
496,2024-12-12,9.990000,AMCR,1.264746,-87.339879
497,2024-12-12,10.390000,F,1.264746,-87.827276
